## IMPORT

In [ ]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
from hypso import Hypso1, Hypso2
sys.path.append(os.path.abspath("D:/Hierarchical Unmixing Label"))
import machi.base as ac

In [ ]:
HYPSO_IMAGE_DIR = r"D:\Downloads"
DATA_DIR        = r"D:\Hierarchical Unmixing Label\hUH\data"
IMAGES_DIR      = DATA_DIR + "\images"

## FUNCTIONS

In [ ]:
#Functions
def plot_rgb_composite(cm, title="", red_band_index=69, green_band_index=46, blue_band_index=26, aspect=0.1, figsize=(10, 10), height=1092, contrast_enhancement=False):
    """
    Create and plot an RGB composite image from a hyperspectral cube.
    
    Parameters:
    - cm: Input hyperspectral data cube or array (can be flattened (h*w, n_bands) or 3D (h, w, n_bands))
    - title: Optional title for the plot
    - red_band_index: Index of the band to use for red channel (default: 69)
    - green_band_index: Index of the band to use for green channel (default: 46)
    - blue_band_index: Index of the band to use for blue channel (default: 26)
    - aspect: Aspect ratio for the plot (default: 0.1)
    - figsize: Figure size as tuple (width, height) in inches (default: (10, 10))
    - height: Height of the image when reshaping from flattened data (default: 1092)
    
    Returns:
    - rgb_image: The processed RGB image
    """
    # Check if input is already a 3D cube or needs reshaping
    if len(cm.shape) == 2:  # Flattened data (h*w, n_bands)
        width = cm.shape[0] // height
        data_cube = cm.reshape(width, height, cm.shape[1])
        data_cube = np.transpose(data_cube, (0, 1, 2))  # Ensure correct orientation
    elif len(cm.shape) == 3:  # Already a cube (h, w, n_bands)
        data_cube = cm
    else:
        raise ValueError(f"Unexpected input shape: {cm.shape}. Expected 2D or 3D array.")
    
    # Extract the specified bands for RGB channels
    red_band = data_cube[:, :, red_band_index]
    green_band = data_cube[:, :, green_band_index]
    blue_band = data_cube[:, :, blue_band_index]

    # Stack the bands to create an RGB image
    rgb_image = np.stack((red_band, green_band, blue_band), axis=-1)
    
    # Process data for better visualization
    # Replace NaN values with 0
    rgb_image[np.isnan(rgb_image)] = 0

    # Apply normalization to each channel
    for i in range(3):
        channel = rgb_image[:,:,i]
        
        # Always use min-max normalization regardless of value range
        min_val = np.nanmin(channel)
        max_val = np.nanmax(channel)
        
        if max_val > min_val:  # Avoid division by zero
            # Normalize to [0,1] range
            channel = (channel - min_val) / (max_val - min_val)
            
            # Apply contrast enhancement if requested
            if contrast_enhancement and np.any(channel > 0):
                # Only enhance contrast if we have enough non-zero values
                non_zero_values = channel[channel > 0]
                if len(non_zero_values) > 10:  # Arbitrary threshold
                    percentiles = np.nanpercentile(channel, [2, 98])
                    p_low, p_high = percentiles[0], percentiles[1]
                    if p_high > p_low:
                        channel = np.clip(channel, p_low, p_high)
                        channel = (channel - p_low) / (p_high - p_low)
            
            rgb_image[:,:,i] = channel
    
    # Final normalization and cleanup
    rgb_image = np.clip(rgb_image, 0, 1)
    
    # Rotate for proper orientation
    rgb_image = np.rot90(rgb_image)
    
    # Create and display the plot
    fig, ax = plt.subplots(1, 1, figsize=figsize)
    if title:
        plt.title(title)
    plt.imshow(rgb_image, aspect=aspect)
    plt.axis('off')  # Hide axes for cleaner visualization
    
    return rgb_image

def hypso_MACHI_Concatenate(hypso_file_name, flat_combined_input, saturated_combined_input, verbose=True, correction="L1D", hypso=1, band_count=112):
    
    # Create the full path to the hypso file
    if os.path.isabs(hypso_file_name):
        # If the provided path is already absolute, use it directly
        hypso_file_path = hypso_file_name
    else:
        # If it's a relative path or just a filename, join with HYPSO_IMAGE_DIR
        # Check if HYPSO_IMAGE_DIR is defined in the environment
        if 'HYPSO_IMAGE_DIR' in globals() or 'HYPSO_IMAGE_DIR' in locals():
            hypso_file_path = os.path.join(HYPSO_IMAGE_DIR, hypso_file_name)
        else:
            # If HYPSO_IMAGE_DIR is not defined, use the provided name as is
            # and issue a warning
            print("Warning: HYPSO_IMAGE_DIR not defined, using provided path as is")
            hypso_file_path = hypso_file_name
    
    # Ensure the file exists
    if not os.path.exists(hypso_file_path):
        raise FileNotFoundError(f"Hypso file not found: {hypso_file_path}")
    
    # Extract image name from file path
    image_name = os.path.splitext(os.path.basename(hypso_file_path))[0]
    if verbose:
        print(f"Processing {image_name}")

    # load and calibrate the hypso image, e.g. convert to radiance
    if hypso==2:
        satobj = Hypso2(path=hypso_file_path, verbose=verbose)
    else:
        satobj = Hypso1(path=hypso_file_path, verbose=verbose)


    if correction == "L1A":
        data = satobj.l1a_cube
    elif correction == "L1B":
        satobj.generate_l1b_cube()
        data = satobj.l1b_cube
    elif correction == "L1C":
        satobj.generate_l1b_cube()
        satobj.generate_l1c_cube()
        data = satobj.l1c_cube
    else:
        satobj.generate_l1b_cube()
        satobj.generate_l1c_cube()
        satobj.generate_l1d_cube()
        data = satobj.l1d_cube

   # Extract spectral bands 
    if band_count==120:
        cube=np.array(data)
    else:
        band_count=112
        cube=np.array(data)[:,:,6:118]

    #Flatten the cube
    flat = cube.reshape(-1,band_count)

    #Atmospheric correction
    T, S, objs = ac.atm_correction(flat, solar=np.ones(band_count), verbose=False, tol=0.01, est_min_R=0.05)
    cube_norm = (cube - S) /T
    
    # Calculate mins on the cube
    mins = [np.nanmin(cube_norm[:,:,i]) for i in range(band_count)]
    cube_norm = cube_norm - 0.95*np.array(mins)
    cube_norm = np.nan_to_num(cube_norm, nan=0.0)
    
    # Flatten the normalized cube
    flat_norm = cube_norm.reshape(-1,band_count)

    # Combine with existing data
    flat_combined = np.concatenate((flat_norm, flat_combined_input), axis=0)

    if verbose:
        print("flat_norm.shape", flat_norm.shape)
        print("flat_combined_input.shape", flat_combined_input.shape)
        print("flat_combined.shape", flat_combined.shape)

    # Identify saturated pixels (values > 35000 in raw data)
    maxes = np.max(np.array(satobj.l1a_cube), axis=-1)
    saturated = maxes > 35000
    saturated_flat = saturated.reshape(-1)
    saturated_flat = np.asarray(saturated_flat, dtype=bool)
    saturated_combined = np.concatenate((saturated_flat, saturated_combined_input), axis=0)

    # Combine correction level with spectral band count to create a complete correction identifier
    correction_with_band_count = f"{correction}_{band_count}"

    # Use the plot_rgb_composite function for visualization
    rgb=plot_rgb_composite(
        cube_norm, 
        title=f"{image_name} ("+correction_with_band_count+"MACHI)",
        red_band_index=69-6,  # Adjust indices to match the 6-118 band range
        green_band_index=46-6,
        blue_band_index=26-6,
        aspect=0.1,
        figsize=(10, 10)
    )
    # Save outputs
    os.makedirs(IMAGES_DIR, exist_ok=True)
    plt.savefig(os.path.join(IMAGES_DIR, f'{image_name}_rgb_composite_{correction_with_band_count}_MACHI.png'))
    np.save(os.path.join(IMAGES_DIR, f'{image_name}_flat_{correction_with_band_count}_MACHI.npy'), flat_norm)

    if verbose:
        print(f"Files saved to: {IMAGES_DIR}")
        print("-"*50+'\n')

    return flat_combined, saturated_combined, flat_norm, satobj

def hypso_Concatenate(hypso_file_name, flat_combined_input, saturated_combined_input, verbose=True, correction="L1A", hypso=1, band_count=120):

    # Create the full path to the hypso file
    if os.path.isabs(hypso_file_name):
        # If the provided path is already absolute, use it directly
        hypso_file_path = hypso_file_name
    else:
        # If it's a relative path or just a filename, join with HYPSO_IMAGE_DIR
        # Check if HYPSO_IMAGE_DIR is defined in the environment
        if 'HYPSO_IMAGE_DIR' in globals() or 'HYPSO_IMAGE_DIR' in locals():
            hypso_file_path = os.path.join(HYPSO_IMAGE_DIR, hypso_file_name)
        else:
            # If HYPSO_IMAGE_DIR is not defined, use the provided name as is
            # and issue a warning
            print("Warning: HYPSO_IMAGE_DIR not defined, using provided path as is")
            hypso_file_path = hypso_file_name
    
    # Ensure the file exists
    if not os.path.exists(hypso_file_path):
        raise FileNotFoundError(f"Hypso file not found: {hypso_file_path}")
    # Extract image name from file path
    image_name = os.path.splitext(os.path.basename(hypso_file_path))[0]
    if verbose:
        print(f"Processing {image_name} using {correction}")
    
    # Load and calibrate the hypso image
    if hypso==2:
        satobj = Hypso2(path=hypso_file_path, verbose=verbose)
    else:
        satobj = Hypso1(path=hypso_file_path, verbose=verbose)
    
    if correction == "L1A":
        data = satobj.l1a_cube
    elif correction == "L1B":
        satobj.generate_l1b_cube()
        data = satobj.l1b_cube
    elif correction == "L1C":
        satobj.generate_l1c_cube()
        data = satobj.l1c_cube
    elif correction == "L1D":
        satobj.generate_l1d_cube()
        data = satobj.l1d_cube
    else:
        print("correction valid, defaulting to L1B")
        correction="L1B"
        data = satobj.l1b_cube

    # Extract spectral bands 6-118
    if band_count==112:
        cube=np.array(data)[:,:,6:118]
    elif band_count==120:
        cube=np.array(data)

    # Flatten the cube
    flat = cube.reshape(-1,band_count)
    
    # Combine with existing data
    flat_combined = np.concatenate((flat, flat_combined_input), axis=0)
    
    # Identify saturated pixels (values > 35000 in raw data)
    maxes = np.max(np.array(satobj.l1a_cube), axis=-1)
    saturated = maxes > 35000
    saturated_flat = saturated.reshape(-1)
    saturated_flat = np.asarray(saturated_flat, dtype=bool)
    saturated_combined = np.concatenate((saturated_flat, saturated_combined_input), axis=0)
    saturated_combined = np.asarray(saturated_combined, dtype=bool)

    # Combine correction level with spectral band count to create a complete correction identifier
    correction_with_band_count = f"{correction}_{band_count}"

    # Use the plot_rgb_composite function for visualization
    rgb=plot_rgb_composite(
        cube, 
        title=f"{image_name} "+correction_with_band_count,
        red_band_index=69-6,  # Adjust indices to match the 6-118 band range
        green_band_index=46-6,
        blue_band_index=26-6,
        aspect=0.1,
        figsize=(10, 10)
    )
    
    # Save outputs 
    os.makedirs(IMAGES_DIR, exist_ok=True)
    plt.savefig(os.path.join(IMAGES_DIR, f'{image_name}_rgb_composite_{correction_with_band_count}.png'))
    np.save(os.path.join(IMAGES_DIR, f'{image_name}_flat_{correction_with_band_count}.npy'), flat)
    
    if verbose:
        print(f"Files saved to: {IMAGES_DIR}")
        print("-"*50+'\n')

    return flat_combined, saturated_combined, flat, satobj

## COMBINE IMAGES

In [ ]:
"""
Removing some spectral bands in corrected images, this is because they have less useful information.
Raw keeps all bands, can be changed but would require changing onboard software.
"""
combined_L1D_MACHI= np.empty((0,112))
combined_L1A= np.empty((0,120))

combined_saturated = np.empty(0).astype(bool)

In [ ]:
combined_L1D_MACHI, combined_saturated, caspiansea1_2025_04_08, satobj_caspiansea1_2025_04_08 =hypso_MACHI_Concatenate("caspiansea1_2025-04-08T07-11-56Z-l1a.nc",combined_L1D_MACHI, combined_saturated)

In [ ]:
combined_L1D_MACHI, combined_saturated, yucatan2_2025_02_06_L1D,            satobj_yucatan2_2025_02_06          =hypso_MACHI_Concatenate("yucatan2_2025-02-06T16-01-18Z-l1a.nc",          combined_L1D_MACHI, combined_saturated)
combined_L1D_MACHI, combined_saturated, kemigawa_2024_12_17_L1D,            satobj_kemigawa_2024_12_17          =hypso_MACHI_Concatenate("kemigawa_2024-12-17T01-01-32Z-l1a.nc",          combined_L1D_MACHI, combined_saturated)
combined_L1D_MACHI, combined_saturated, chapala_2025_02_24_L1D,             satobj_chapala_2025_02_24           =hypso_MACHI_Concatenate("chapala_2025-02-24T16-52-47Z-l1a.nc",           combined_L1D_MACHI, combined_saturated)
combined_L1D_MACHI, combined_saturated, grizzlybay_2025_01_27_L1D,          satobj_grizzlybay_2025_01_27        =hypso_MACHI_Concatenate("grizzlybay_2025-01-27T18-19-56Z-l1a.nc",        combined_L1D_MACHI, combined_saturated)
combined_L1D_MACHI, combined_saturated, victoriaLand_2025_02_07_L1D,        satobj_victoriaLand_2025_02_07      =hypso_MACHI_Concatenate("victoriaLand_2025-02-07T20-35-33Z-l1a.nc",      combined_L1D_MACHI, combined_saturated)
combined_L1D_MACHI, combined_saturated, catala_2025_01_28_L1D,              satobj_catala_2025_01_28            =hypso_MACHI_Concatenate("catala_2025-01-28T19-17-32Z-l1a.nc",            combined_L1D_MACHI, combined_saturated)
combined_L1D_MACHI, combined_saturated, khnifiss_2025_02_12_L1D,            satobj_khnifiss_2025_02_12          =hypso_MACHI_Concatenate("khnifiss_2025-02-12T11-05-35Z-l1a.nc",          combined_L1D_MACHI, combined_saturated)
combined_L1D_MACHI, combined_saturated, menindee_2025_02_18_L1D,            satobj_menindee_2025_02_18          =hypso_MACHI_Concatenate("menindee_2025-02-18T00-10-42Z-l1a.nc",          combined_L1D_MACHI, combined_saturated)
combined_L1D_MACHI, combined_saturated, tampa_2024_11_12_L1D,               satobj_tampa_2024_11_12             =hypso_MACHI_Concatenate("tampa_2024-11-12T15-31-55Z-l1a.nc",             combined_L1D_MACHI, combined_saturated)
combined_L1D_MACHI, combined_saturated, falklandsatlantic_2024_12_18_L1D,   satobj_falklandsatlantic_2024_12_18 =hypso_MACHI_Concatenate("falklandsatlantic_2024-12-18T13-25-18Z-l1a.nc", combined_L1D_MACHI, combined_saturated)

In [ ]:
combined_L1A, combined_saturated, yucatan2_2025_02_06_L1A,            satobj_yucatan2_2025_02_06          =hypso_Concatenate("yucatan2_2025-02-06T16-01-18Z-l1a.nc",          combined_L1A, combined_saturated)
combined_L1A, combined_saturated, kemigawa_2024_12_17_L1A,            satobj_kemigawa_2024_12_17          =hypso_Concatenate("kemigawa_2024-12-17T01-01-32Z-l1a.nc",          combined_L1A, combined_saturated)
combined_L1A, combined_saturated, chapala_2025_02_24_L1A,             satobj_chapala_2025_02_24           =hypso_Concatenate("chapala_2025-02-24T16-52-47Z-l1a.nc",           combined_L1A, combined_saturated)
combined_L1A, combined_saturated, grizzlybay_2025_01_27_L1A,          satobj_grizzlybay_2025_01_27        =hypso_Concatenate("grizzlybay_2025-01-27T18-19-56Z-l1a.nc",        combined_L1A, combined_saturated)
combined_L1A, combined_saturated, victoriaLand_2025_02_07_L1A,        satobj_victoriaLand_2025_02_07      =hypso_Concatenate("victoriaLand_2025-02-07T20-35-33Z-l1a.nc",      combined_L1A, combined_saturated)
combined_L1A, combined_saturated, catala_2025_01_28_L1A,              satobj_catala_2025_01_28            =hypso_Concatenate("catala_2025-01-28T19-17-32Z-l1a.nc",            combined_L1A, combined_saturated)
combined_L1A, combined_saturated, khnifiss_2025_02_12_L1A,            satobj_khnifiss_2025_02_12          =hypso_Concatenate("khnifiss_2025-02-12T11-05-35Z-l1a.nc",          combined_L1A, combined_saturated)
combined_L1A, combined_saturated, menindee_2025_02_18_L1A,            satobj_menindee_2025_02_18          =hypso_Concatenate("menindee_2025-02-18T00-10-42Z-l1a.nc",          combined_L1A, combined_saturated)
combined_L1A, combined_saturated, tampa_2024_11_12_L1A,               satobj_tampa_2024_11_12             =hypso_Concatenate("tampa_2024-11-12T15-31-55Z-l1a.nc",             combined_L1A, combined_saturated)
combined_L1A, combined_saturated, falklandsatlantic_2024_12_18_L1A,   satobj_falklandsatlantic_2024_12_18 =hypso_Concatenate("falklandsatlantic_2024-12-18T13-25-18Z-l1a.nc", combined_L1A, combined_saturated)

## SAVE

In [ ]:
np.save(f'{IMAGES_DIR}\\combined_10_L1D_112_MACHI.npy', combined_L1D_MACHI)
np.save(f'{IMAGES_DIR}\\combined_10_saturated.npy', combined_saturated)

In [ ]:
np.save(f'{IMAGES_DIR}\\combined_10_L1A_120.npy', combined_L1A)
np.save(f'{IMAGES_DIR}\\combined_10_saturated.npy', combined_saturated)

## LOAD

In [ ]:
combined_L1A       = np.load(f'{IMAGES_DIR}\\combined_10_L1A_120.npy')
combined_L1D_MACHI = np.load(f'{IMAGES_DIR}\\combined_10_L1D_112_MACHI.npy')
combined_saturated = np.load(f'{IMAGES_DIR}\\combined_10_saturated.npy')
print(combined_L1A.shape)
print(combined_L1D_MACHI.shape)
print(combined_saturated.shape)

In [ ]:
yucatan2_2025_02_06_L1D         = np.load(f'{IMAGES_DIR}\\yucatan2_2025-02-06T16-01-18Z-l1a_flat_L1D_112_MACHI.npy')
kemigawa_2024_12_17_L1D         = np.load(f'{IMAGES_DIR}\\kemigawa_2024-12-17T01-01-32Z-l1a_flat_L1D_112_MACHI.npy')
chapala_2025_02_24_L1D          = np.load(f'{IMAGES_DIR}\\chapala_2025-02-24T16-52-47Z-l1a_flat_L1D_112_MACHI.npy')
grizzlybay_2025_01_27_L1D       = np.load(f'{IMAGES_DIR}\\grizzlybay_2025-01-27T18-19-56Z-l1a_flat_L1D_112_MACHI.npy')
victoriaLand_2025_02_07_L1D     = np.load(f'{IMAGES_DIR}\\victoriaLand_2025-02-07T20-35-33Z-l1a_flat_L1D_112_MACHI.npy')
catala_2025_01_28_L1D           = np.load(f'{IMAGES_DIR}\\catala_2025-01-28T19-17-32Z-l1a_flat_L1D_112_MACHI.npy')
khnifiss_2025_02_12_L1D         = np.load(f'{IMAGES_DIR}\\khnifiss_2025-02-12T11-05-35Z-l1a_flat_L1D_112_MACHI.npy')
menindee_2025_02_18_L1D         = np.load(f'{IMAGES_DIR}\\menindee_2025-02-18T00-10-42Z-l1a_flat_L1D_112_MACHI.npy')
tampa_2024_11_12_L1D            = np.load(f'{IMAGES_DIR}\\tampa_2024-11-12T15-31-55Z-l1a_flat_L1D_112_MACHI.npy')
falklandsatlantic_2024_12_18_L1D= np.load(f'{IMAGES_DIR}\\falklandsatlantic_2024-12-18T13-25-18Z-l1a_flat_L1D_112_MACHI.npy')

## PLOT

In [ ]:
rgb_maci_combined = plot_rgb_composite(combined_L1D_MACHI, "combined_L1D_MACHI")

In [ ]:
rgb_yucatan2_2025_02_06_L1D     = plot_rgb_composite(yucatan2_2025_02_06_L1D, "yucatan2_2025_02_06")
rgb_kemigawa_2024_12_17_L1D     = plot_rgb_composite(kemigawa_2024_12_17_L1D, "kemigawa_2024_12_17")
rgb_chapala_2025_02_24_L1D      = plot_rgb_composite(chapala_2025_02_24_L1D, "chapala_2025_02_24")
rgb_grizzlybay_2025_01_27_L1D   = plot_rgb_composite(grizzlybay_2025_01_27_L1D, "grizzlybay_2025_01_27")
rgb_victoriaLand_2025_02_07_L1D = plot_rgb_composite(victoriaLand_2025_02_07_L1D, "victoriaLand_2025_02_07")
rgb_catala_2025_01_28_L1D       = plot_rgb_composite(catala_2025_01_28_L1D, "catala_2025_01_28")
rgb_khnifiss_2025_02_12_L1D     = plot_rgb_composite(khnifiss_2025_02_12_L1D, "khnifiss_2025_02_12")
rgb_menindee_2025_02_18_L1D     = plot_rgb_composite(menindee_2025_02_18_L1D, "menindee_2025_02_18")
rgb_tampa_2024_11_12_L1D        = plot_rgb_composite(tampa_2024_11_12_L1D, "tampa_2024_11_12")
rgb_falklandsatlantic_2024_12_18_L1D = plot_rgb_composite(falklandsatlantic_2024_12_18_L1D, "falklandsatlantic_2024_12_18")

In [ ]:
rgb_yucatan2_2025_02_06_L1A     = plot_rgb_composite(yucatan2_2025_02_06_L1A, "yucatan2_2025_02_06")
rgb_kemigawa_2024_12_17_L1A     = plot_rgb_composite(kemigawa_2024_12_17_L1A, "kemigawa_2024_12_17")
rgb_chapala_2025_02_24_L1A      = plot_rgb_composite(chapala_2025_02_24_L1A, "chapala_2025_02_24")
rgb_grizzlybay_2025_01_27_L1A   = plot_rgb_composite(grizzlybay_2025_01_27_L1A, "grizzlybay_2025_01_27")
rgb_victoriaLand_2025_02_07_L1A = plot_rgb_composite(victoriaLand_2025_02_07_L1A, "victoriaLand_2025_02_07")
rgb_catala_2025_01_28_L1A       = plot_rgb_composite(catala_2025_01_28_L1A, "catala_2025_01_28")
rgb_khnifiss_2025_02_12_L1A     = plot_rgb_composite(khnifiss_2025_02_12_L1A, "khnifiss_2025_02_12")
rgb_menindee_2025_02_18_L1A     = plot_rgb_composite(menindee_2025_02_18_L1A, "menindee_2025_02_18")
rgb_tampa_2024_11_12_L1A        = plot_rgb_composite(tampa_2024_11_12_L1A, "tampa_2024_11_12")
rgb_falklandsatlantic_2024_12_18_L1A = plot_rgb_composite(falklandsatlantic_2024_12_18_L1A, "falklandsatlantic_2024_12_18")

## DEBUG

In [ ]:
#empty